<a href="https://colab.research.google.com/github/Ipaye/digital-o/blob/master/SIMPE_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

In [ ]:
# wrap the output in colab cells
from IPython.display import HTML, display

def set_css():
  display(HTML('''

  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!nvidia-smi

Fri Nov 10 13:28:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
! pip install simplet5 -q

In [ ]:
dataset_books = load_dataset("pszemraj/booksum-short")
dataset_books

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'chapter', 'chapter_length', 'summary_name', 'summary_url', 'summary_text', 'summary_analysis', 'summary_length', 'analysis_length'],
        num_rows: 5912
    })
    validation: Dataset({
        features: ['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'chapter', 'chapter_length', 'summary_name', 'summary_url', 'summary_text', 'summary_analysis', 'summary_length', 'analysis_length'],
        num_rows: 1012
    })
    test: Dataset({
        features: ['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'chapter', 'chapter_length', 'summary_name', 'summary_url', 'summary_text', 'summary_analysis', 'summary_length', 'analysis_length'],
        num_rows: 988
    })
})

In [ ]:
dataset_books['train']

Dataset({
    features: ['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'chapter', 'chapter_length', 'summary_name', 'summary_url', 'summary_text', 'summary_analysis', 'summary_length', 'analysis_length'],
    num_rows: 5912
})

In [ ]:
ddataset = pd.DataFrame(dataset_books['train'])

In [ ]:
ddataset

,bid,is_aggregate,source,chapter_path,summary_path,book_id,summary_id,content,summary,chapter,chapter_length,summary_name,summary_url,summary_text,summary_analysis,summary_length,analysis_length
0,27681,False,cliffnotes,all_chapterized_books/27681-chapters/03.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 3,chapter 3,None,"{""name"": ""Chapter 3"", ""url"": ""https://web.arch...","\n ""Before these fields were shorn and tilled...",4938,Chapter 3,https://web.archive.org/web/20201101053205/htt...,In another part of the forest by the river a f...,This chapter introduces the other three main a...,299,149
1,27681,False,cliffnotes,all_chapterized_books/27681-chapters/04.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 4,chapter 4,None,"{""name"": ""Chapter 4"", ""url"": ""https://web.arch...","\n ""Well, go thy way: thou shalt not from thi...",4897,Chapter 4,https://web.archive.org/web/20201101053205/htt...,When the mounted party from Fort Howard approa...,Since this chapter is mostly one of surface ac...,463,75
2,27681,False,cliffnotes,all_chapterized_books/27681-chapters/05.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 5,chapter 5,None,"{""name"": ""Chapter 5"", ""url"": ""https://web.arch...","\n ""In such a night\n Di...",5290,Chapter 5,https://web.archive.org/web/20201101053205/htt...,"The pursuit of Magua is unsuccessful, but Hawk...",Here the reader encounters the first bloodshed...,488,156
3,27681,False,cliffnotes,all_chapterized_books/27681-chapters/06.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 6,chapter 6,None,"{""name"": ""Chapter 6"", ""url"": ""https://web.arch...","\n ""Those strains that once did sweet in Zion...",6158,Chapter 6,https://web.archive.org/web/20201101053205/htt...,Heyward and the girls are uneasy and Gamut is ...,This chapter shows Cooper in his most inventiv...,463,128
4,27681,False,cliffnotes,all_chapterized_books/27681-chapters/09.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 9,chapter 9,None,"{""name"": ""Chapter 9"", ""url"": ""https://web.arch...","\n ""Be gay securely...",4842,Chapter 9,https://web.archive.org/web/20201101053205/htt...,"In the stillness that follows, Heyward finds i...","With the woodsmen off the scene of action, thi...",388,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5907,2833,False,pinkmonkey,all_chapterized_books/2833-chapters/44.txt,finished_summaries/pinkmonkey/Portrait of a La...,Portrait of a Lady.chapter 44,chapter 44,None,"{""name"": ""Chapter 44"", ""url"": ""https://web.arc...",\nThe Countess Gemini was often extremely bore...,8298,Chapter 44,https://web.archive.org/web/20180820033733/htt...,The bored Countess Gemini is happily surprised...,Notes It seems that whenever James wants to br...,432,97
5908,2833,False,pinkmonkey,all_chapterized_books/2833-chapters/46.txt,finished_summaries/pinkmonkey/Portrait of a La...,Portrait of a Lady.chapter 46,chapter 46,None,"{""name"": ""Chapter 46"", ""url"": ""https://web.arc...",\nLord Warburton was not seen in Mrs. Osmond's...,6091,Chapter 46,https://web.archive.org/web/20180820033733/htt...,Lord Warburton stays away from the Osmonds for...,Notes The ugly scene between Isabel and Gilber...,449,89
5909,2833,False,pinkmonkey,all_chapterized_books/2833-chapters/52.txt,finished_summaries/pinkmonkey/Portrait of a La...,Portrait of a Lady.chapter 52,chapter 52,None,"{""name"": ""Chapter 52"", ""url"": ""https://web.arc...",\nThere was a train for Turin and Paris that e...,5820,Chapter 52,https://web.archive.org/web/20180820033733/htt...,"Before Isabel leaves Rome, she goes to see Pan...",Notes The visit to the convent reveals the nex...,450,269
5910,2833,False,pinkmonkey,all_chapterized_books/2833-chapters/53.txt,finished_summaries/pinkmonkey/Portrait of a La...,Portrait of a Lady.chapter 53,chapter 53,No

In [ ]:
ddataset = ddataset.rename(columns={"chapter":"source_text", "summary_text":"target_text"})

In [ ]:
ddataset = ddataset[['source_text', 'target_text']]

In [ ]:
import re

In [ ]:
ddataset['source_text'] = ddataset['source_text'].apply(lambda x: re.sub('\W+','', x))

In [ ]:
ddataset.head()

,source_text,target_text
0,"\n ""Before these fields were shorn and tilled...",In another part of the forest by the river a f...
1,"\n ""Well, go thy way: thou shalt not from thi...",When the mounted party from Fort Howard approa...
2,"\n ""In such a night\n Di...","The pursuit of Magua is unsuccessful, but Hawk..."
3,"\n ""Those strains that once did sweet in Zion...",Heyward and the girls are uneasy and Gamut is ...
4,"\n ""Be gay securely...","In the stillness that follows, Heyward finds i..."


In [ ]:
ddataset['source_text'] = "summarize: " + ddataset['source_text']

In [ ]:
ddataset.head()

,source_text,target_text
0,"summarize: \n ""Before these fields were shorn...",In another part of the forest by the river a f...
1,"summarize: \n ""Well, go thy way: thou shalt n...",When the mounted party from Fort Howard approa...
2,"summarize: \n ""In such a ...","The pursuit of Magua is unsuccessful, but Hawk..."
3,"summarize: \n ""Those strains that once did sw...",Heyward and the girls are uneasy and Gamut is ...
4,"summarize: \n ""Be g...","In the stillness that follows, Heyward finds i..."


In [ ]:
train_df, test_df = train_test_split(ddataset, test_size=0.3)
train_df.shape, test_df.shape

((4138, 2), (1774, 2))

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100],
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=8, max_epochs=5, use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
! ( cd outputs; ls )

simplet5-epoch-0-train-loss-3.314-val-loss-2.9552
simplet5-epoch-1-train-loss-3.0077-val-loss-2.8972
simplet5-epoch-2-train-loss-2.8165-val-loss-2.8946
simplet5-epoch-3-train-loss-2.6562-val-loss-2.8827
simplet5-epoch-4-train-loss-2.5119-val-loss-2.9079


In [ ]:
model.load_model("t5","outputs/simplet5-epoch-4-train-loss-2.5119-val-loss-2.9079", use_gpu=True)

In [ ]:
text_to_summarize="""
OpenAI announced its new, more powerful GPT-4 Turbo artificial intelligence model Monday during its first in-person event, and revealed a new option that will let users create custom versions of its viral ChatGPT chatbot. It’s also cutting prices on the fees that companies and developers pay to run its software.

OpenAI’s announcements show that one of the hottest companies in tech is rapidly evolving its offerings in an effort to stay ahead of rivals like Anthropic, Google
 and Meta
 in the AI arms race. ChatGPT, which broke records as the fastest-growing consumer app in history months after its launch, now has about 100 million weekly active users, OpenAI said Monday. More than 92% of Fortune 500 companies use the platform, up from 80% in August, and they span across industries like financial services, legal applications and education, OpenAI CTO Mira Murati told reporters Monday.

The event also included a surprise appearance by Microsoft CEO Satya Nadella.

“The systems that are needed as you aggressively push forward on your road map require us to be on the top of our game, and we intend fully to commit ourselves fully to making sure you all... have not only the best systems for training and inference, but also the most compute,” Nadella told OpenAI CEO Sam Altman while onstage together. He added, “That’s the way we’re going to make progress.”

Earlier this year, Microsoft’s expanded investment in OpenAI — an additional $10 billion — made it the biggest AI investment of the year, according to PitchBook, and in April, the startup reportedly closed a $300 million share sale at a valuation between $27 billion and $29 billion, with investments from firms such as Sequoia Capital and Andreessen Horowitz. As recently as last month, OpenAI was reportedly in talks to close a deal that would lead to an $80 billion valuation.

In his speech Monday, Altman said the day’s announcements came from conversations with developers about their needs over the past year. And when it comes to GPT-5, Altman told reporters, “We want to do it, but we don’t have a timeline.”

Here’s what OpenAI announced Monday:

GPT-4 Turbo is the latest AI model, and it now provides answers with context up to April 2023. Prior versions were cut off at January 2022. For example, if you asked GPT-4 who won the Super Bowl in February 2022, it wouldn’t have been able to tell you. GPT-4 Turbo can.

“We are just as annoyed as all of you, probably more, that GPT’s knowledge about the world ended in 2021,” Altman said in a speech Monday.

It also accepts a lot more input. While earlier versions limited you to about 3,000 words, the GPT-4 Turbo accepts inputs of up to 300 pages in length. It means you could ask it to summarize entire books.

GPT-4 also supports DALL-E 3 AI-generated images and text-to-speech. It also has six preset voices to choose from, so you can choose to hear the answer to a query in a variety of different voices.

OpenAI said GPT-4 Turbo is available in preview for developers now and will be released to all in the coming weeks.

OpenAI said it’s also cutting the prices for developers. “GPT-4 Turbo input tokens are 3x cheaper than GPT-4 at $0.01 and output tokens are 2x cheaper at $0.03,” the company said, which means companies and developers should save more when running lots of information through the AI models.

Until now, ChatGPT’s enterprise and business offerings were the only way people could upload their own data to train and customize the chatbot for particular industries and use cases. Now it’s adding the option for anyone to create custom chatbots.

AI “agents” are one of the buzziest uses of the technology recently, with many startups vying to offer the kind of personalized AI tools that consumers may already be familiar with via pop culture representations, such as Tony Stark’s J.A.R.V.I.S. in Marvel movies, or Pam in Disney Channel’s Smart House.

“Anyone can easily build their own GPT—no coding is required,” the company wrote in a release. “You can make them for yourself, just for your company’s internal use, or for everyone. Creating one is as easy as starting a conversation, giving it instructions and extra knowledge, and picking what it can do, like searching the web, making images or analyzing data.”

More than two million developers building their own tools using ChatGPT’s API will also be able to customize the chatbot, meaning consumers will likely see personalized AI chatbots popping up in many more places, including apps and websites they use regularly.

Now that users and developers can launch their own, personalized AI chatbots, OpenAI is introducing a new revenue driver for the company: Its own version of the app store.

The GPT Store allows people who create their own GPTs to make them available for public download, and in the coming months, OpenAI said people will be able to earn money based on their creation’s usage numbers.

“Once in the store, GPTs become searchable and may climb the leaderboards,” the company wrote in a release. “We will also spotlight the most useful and delightful GPTs we come across in categories like productivity, education, and ‘just for fun.’”

As for revenue share for people who create custom chatbots featured in the store, the company will start with “just sharing a part of the subscription revenue overall,” Altman told reporters Monday. Right now, the company is planning to base the payout on active users plus category bonuses, and may support subscriptions for specific GPTs later.

“What OpenAI is really in the business of selling is intelligence -- and that, and intelligent agents, is really where it will trend over time,” Altman told reporters.

New all-in-one image-generation, browsing and summarization
Until Monday, ChatGPT users had to hop between different apps and websites to use all of OpenAI’s tools, which contributed to a slightly higher learning curve. On Monday, the company announced it has streamlined its AI tools into one place: Using ChatGPT now offers image generation via DALL-E, browsing, data analysis, document upload and PDF search. Before now, Anthropic’s Claude was the only competitor chatbot to allow PDF search.

Copyright shield
As generative AI-related legal action is on the rise, Altman announced Monday that OpenAI will “step in and defend our customers” and “pay the costs incurred if you face legal claims around copyright infringement.” It echoes similar statements made by Google, Microsoft and Adobe.
"""

In [ ]:
model_t5 =  "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(model_t5)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
sentences = nltk.tokenize.sent_tokenize(text_to_summarize)
sentences

['\nOpenAI announced its new, more powerful GPT-4 Turbo artificial intelligence model Monday during its first in-person event, and revealed a new option that will let users create custom versions of its viral ChatGPT chatbot.',
 'It’s also cutting prices on the fees that companies and developers pay to run its software.',
 'OpenAI’s announcements show that one of the hottest companies in tech is rapidly evolving its offerings in an effort to stay ahead of rivals like Anthropic, Google\n and Meta\n in the AI arms race.',
 'ChatGPT, which broke records as the fastest-growing consumer app in history months after its launch, now has about 100 million weekly active users, OpenAI said Monday.',
 'More than 92% of Fortune 500 companies use the platform, up from 80% in August, and they span across industries like financial services, legal applications and education, OpenAI CTO Mira Murati told reporters Monday.',
 'The event also included a surprise appearance by Microsoft CEO Satya Nadella.',

In [ ]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

100

In [ ]:
len(text_to_summarize)

6484

In [ ]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

2

In [ ]:
[len(tokenizer.tokenize(c)) for c in chunks]

[965, 521]

In [ ]:
chunks

['OpenAI announced its new, more powerful GPT-4 Turbo artificial intelligence model Monday during its first in-person event, and revealed a new option that will let users create custom versions of its viral ChatGPT chatbot. It’s also cutting prices on the fees that companies and developers pay to run its software. OpenAI’s announcements show that one of the hottest companies in tech is rapidly evolving its offerings in an effort to stay ahead of rivals like Anthropic, Google\n and Meta\n in the AI arms race. ChatGPT, which broke records as the fastest-growing consumer app in history months after its launch, now has about 100 million weekly active users, OpenAI said Monday. More than 92% of Fortune 500 companies use the platform, up from 80% in August, and they span across industries like financial services, legal applications and education, OpenAI CTO Mira Murati told reporters Monday. The event also included a surprise appearance by Microsoft CEO Satya Nadella. “The systems that are n

In [ ]:
len(chunks)

2

In [ ]:
model.predict(chunks[0])

['In a speech to reporters, Altman says that the new AI model is more powerful than GPT-4 and that it can now answer questions with context up to April 2023. It also allows developers to create their own versions of chatbots without having to code.']

In [ ]:
model.predict(chunks[1])

['OpenAI is introducing a new revenue driver: its own version of the app store. People who create their own GPTs will be able to earn money based on their usage numbers. The company also plans to offer subscriptions for certain GPTs']